In [1]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 2.1 MB/s 
     |████████████████████████████████| 16.7 MB 343 kB/s 
     |████████████████████████████████| 6.3 MB 36.8 MB/s 


In [2]:
import pandas as pd
import geopandas as gpd

import folium
from folium import Choropleth
from folium.plugins import HeatMap

In [3]:
mapa = folium.Map(location=[42.32, -71.0589], tiles='openstreetmap', zoom_start=10)
mapa

In [4]:
plate_boundaries = gpd.read_file("/content/Plate_Boundaries.shp")
plate_boundaries['coordinates'] = plate_boundaries.apply(lambda x: [(b,a) for (a,b) in list(x.geometry.coords)], axis='columns')
plate_boundaries.drop('geometry', axis=1, inplace=True)

In [5]:
plate_boundaries.head()

,HAZ_PLATES,HAZ_PLAT_1,HAZ_PLAT_2,Shape_Leng,coordinates
0,TRENCH,SERAM TROUGH (ACTIVE),6722,5.843467,"[(-5.444200361999947, 133.6808931800001), (-5...."
1,TRENCH,WETAR THRUST,6722,1.829013,"[(-7.760600482999962, 125.47879802900002), (-7..."
2,TRENCH,TRENCH WEST OF LUZON (MANILA TRENCH) NORTHERN ...,6621,6.743604,"[(19.817899819000047, 120.09999798800004), (19..."
3,TRENCH,BONIN TRENCH,9821,8.329381,"[(26.175899215000072, 143.20620700100005), (26..."
4,TRENCH,NEW GUINEA TRENCH,8001,11.998145,"[(0.41880004000006466, 132.8273013480001), (0...."


In [6]:
plate_boundaries.shape

(360, 5)

In [7]:
terremotos = pd.read_csv("/content/earthquakes1970-2014.csv", parse_dates=["DateTime"])
terremotos.head()

,DateTime,Latitude,Longitude,Depth,Magnitude,MagType,NbStations,Gap,Distance,RMS,Source,EventID
0,1970-01-04 17:00:40.200,24.139,102.503,31.0,7.5,Ms,90.0,NaN,NaN,0.0,NEI,1.970010e+09
1,1970-01-06 05:35:51.800,-9.628,151.458,8.0,6.2,Ms,85.0,NaN,NaN,0.0,NEI,1.970011e+09
2,1970-01-08 17:12:39.100,-34.741,178.568,179.0,6.1,Mb,59.0,NaN,NaN,0.0,NEI,1.970011e+09
3,1970-01-10 12:07:08.600,6.825,126.737,73.0,6.1,Mb,91.0,NaN,NaN,0.0,NEI,1.970011e+09
4,1970-01-16 08:05:39.000,60.280,-152.660,85.0,6.0,ML,0.0,NaN,NaN,NaN,AK,NaN


In [8]:
terremotos.shape

(5304, 12)

In [9]:
terremotos.isnull().sum()

DateTime         0
Latitude         0
Longitude        0
Depth            0
Magnitude        0
MagType          0
NbStations      54
Gap           5132
Distance      5134
RMS             40
Source           0
EventID         62
dtype: int64

In [10]:
terremotos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5304 entries, 0 to 5303
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   DateTime    5304 non-null   datetime64[ns]
 1   Latitude    5304 non-null   float64       
 2   Longitude   5304 non-null   float64       
 3   Depth       5304 non-null   float64       
 4   Magnitude   5304 non-null   float64       
 5   MagType     5304 non-null   object        
 6   NbStations  5250 non-null   float64       
 7   Gap         172 non-null    float64       
 8   Distance    170 non-null    float64       
 9   RMS         5264 non-null   float64       
 10  Source      5304 non-null   object        
 11  EventID     5242 non-null   float64       
dtypes: datetime64[ns](1), float64(9), object(2)
memory usage: 497.4+ KB


In [11]:
import sys
print(sys.getrecursionlimit())

1000


In [12]:
sys.setrecursionlimit(5000)

In [13]:
terremotos_lista = terremotos[['Latitude', 'Longitude']].values.tolist()

In [14]:
mapa = folium.Map(location=[35, 136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
  folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(mapa)


HeatMap(data=terremotos_lista, radius=15).add_to(mapa)
mapa.save("mapa.html")

In [15]:
mapa2 = folium.Map(location=[35, 136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
  folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(mapa2)


#HeatMap(data=terremotos_lista, radius=15).add_to(mapa)
#mapa.save("mapa.html")

In [16]:
def color_producer(val):
  if val < 50:
    return 'foretsgreen'
  elif val < 100:
    return 'darkorange'
  else:
    return 'darkred'

for i in range(0, len(terremotos)):
  folium.Circle(location=[terremotos.iloc[i]['Latitude'], terremotos.iloc[i]['Longitude']], radius=2000, color=color_producer(terremotos.iloc[i]['Depth'])).add_to(mapa2)

mapa2.save("mapa2.html")

In [18]:
prefeituras = gpd.read_file("/content/japan-prefecture-boundaries.shp")
prefeituras.set_index('prefecture', inplace=True)
prefeituras.head()

,geometry
prefecture,
Aichi,"MULTIPOLYGON (((137.09523 34.65330, 137.09546 ..."
Akita,"MULTIPOLYGON (((139.55725 39.20330, 139.55765 ..."
Aomori,"MULTIPOLYGON (((141.39860 40.92472, 141.39806 ..."
Chiba,"MULTIPOLYGON (((139.82488 34.98967, 139.82434 ..."
Ehime,"MULTIPOLYGON (((132.55859 32.91224, 132.55904 ..."


In [19]:
populacao = pd.read_csv("/content/japan-prefecture-population.csv")
populacao.set_index('prefecture', inplace=True)

In [20]:
populacao.head()

,population
prefecture,
Tokyo,12868000
Kanagawa,8943000
Osaka,8801000
Aichi,7418000
Saitama,7130000


In [21]:
area_sqkm = pd.Series(prefeituras.geometry.to_crs(epsg=32654).area/10**6, name='area_sqkm')
stats = populacao.join(area_sqkm)

stats['density'] = stats['population']/stats['area_sqkm']
stats.head()

,population,area_sqkm,density
prefecture,,,
Tokyo,12868000,1800.614782,7146.448049
Kanagawa,8943000,2383.038975,3752.771186
Osaka,8801000,1923.151529,4576.342460
Aichi,7418000,5164.400005,1436.372085
Saitama,7130000,3794.036890,1879.264806


In [22]:
mapa3 = folium.Map(location=[35, 136], tiles='cartodbpositron', zoom_start=5)
Choropleth(geo_data=prefeituras['geometry'].__geo_interface__, data=stats['density'], key_on='feature.id', fill_color='YlGnBu', 
           legend_name='Densidade demográfica (por km²').add_to(mapa3)

mapa3.save('mapa3.html')

In [23]:
mapa4 = folium.Map(location=[35, 136], tiles='cartodbpositron', zoom_start=5)

def color_producer(magnitude):
  if magnitude > 6.5:
    return 'red'

  else:
    return 'green'

Choropleth(geo_data=prefeituras['geometry'].__geo_interface__, data=stats['density'], key_on='feature.id', fill_color='BuPu',
           legend_name='Densidade demográfica (por km²').add_to(mapa4)

for i in range(0, len(terremotos)):
  folium.Circle(location=[terremotos.iloc[i]['Latitude'], terremotos.iloc[i]['Longitude']],
                popup =("{} {}").format(terremotos.iloc[i]['Magnitude'],
                                        terremotos.iloc[i]['DateTime'].year),
                radius=terremotos.iloc[i]['Magnitude']**5.5,
                color=color_producer(terremotos.iloc[i]['Magnitude'])).add_to(mapa4)

mapa4.save("mapa4.html")